##Data Preparation and Integration

###Primary Key Identification
1. Analyze both datasets and identify a Primary Key for integration.
2. Justify your selection and highlight potential challenges in identifying a unique
key.

In [ ]:
!pip install openpyxl
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/My Drive/Data for Task 2..xlsx"
df = pd.read_excel(file_path)

df1 = pd.read_excel(file_path,sheet_name= 'Work Order Data')
df2 = pd.read_excel(file_path,sheet_name= 'Repair Data')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Analysis of Primary Key for Integration:
Both datasets (Work Order Data and Repair Data) share three key columns:
- **Primary Key**
- **Order No**
- **Segment Number**

1. Identifying a Primary Key
The "Primary Key" column appears in both datasets and has:
- 500 entries in both datasets (No missing values).
- Unique-looking values (e.g., "SO0005588-1") that seem to follow a structured format (likely combining "Order No" and "Segment Number").

Thus, "Primary Key" is the best candidate for integration, as it exists in both datasets and appears to uniquely identify each row.

2. Justification for Selection
- Consistency: "Primary Key" is present in both datasets with the same formatting.
- Uniqueness: Likely derived from "Order No" and "Segment Number," making it specific to each work order.
- Integration Feasibility: Both datasets have the same number of entries (500), meaning there's a one-to-one relationship.

##Data Cleaning
1. Load the datasets using Python.
2. Inspect the column structure, and clean the data:
    * Handle missing values, duplicates
    * Format Correction - Consistent data types across dataset   
    * Apply language translation if applicable.      
3. Provide a brief summary of your data cleaning process.


Summary of Data Cleaning Process:

1. Handled Missing Values -> Replaced missing values in:
          - "Cause" (set to "Unknown")
          - "Correction" (set to "Not Provided")
          - "Coverage" (set to "Not Available")
          - "Actual Hours" (filled with median value).

2. Removed Duplicates-> Dropped 2 duplicate rows from df2 (Repair Data).

3. Ensured Consistent Data Types:
      - Converted "Revenue" and "Cost" to numeric.
      - Transformed "Invoice Date" to datetime format.

In [ ]:
# Find duplicate rows before dropping
duplicates = df2[df2.duplicated(keep=False)]  # Keep=False shows all duplicate occurrences
print("Duplicate Rows:\n", duplicates)


Duplicate Rows:
      Primary Key   Order No  Segment Number Coverage  Qty Part Manufacturer  \
270  SO0059080-2  SO0059080               2      NaN    1              PASE   
271  SO0059080-2  SO0059080               2      NaN    1              PASE   
326  SO0059284-4  SO0059284               4      NaN    1  DEVRE ENTERPRISE   
327  SO0059284-4  SO0059284               4      NaN    1  DEVRE ENTERPRISE   

                             Part Number       Part Description  Revenue  \
270    87630098                     CASE              ELBOW  45   5.5499   
271    87630098                     CASE              ELBOW  45   5.5499   
326  BOLT-Q                       TRINIT  BOLT BY QUARTER POUND   5.1399   
327  BOLT-Q                       TRINIT  BOLT BY QUARTER POUND   5.1399   

     Cost  Invoice Date  Actual Hours Segment Total $  
270  3.1$         45219       12.8897      2612.8196$  
271  3.1$         45219       12.8897      2612.8196$  
326  1.3$         45286       43.7592 

In [ ]:
# Handling missing values

# In df1 (Work Order Data):
df1["Cause"].fillna("Unknown", inplace=True)  # Replace missing 'Cause' with 'Unknown'
df1["Correction"].fillna("Not Provided", inplace=True)  # Replace missing 'Correction' with 'Not Provided'

# In df2 (Repair Data):
df2["Coverage"].fillna("Not Available", inplace=True)  # Replace missing 'Coverage' with 'Not Available'
df2["Actual Hours"].fillna(df2["Actual Hours"].median(), inplace=True)  # Replace missing 'Actual Hours' with median value

# Removing duplicates
df2.drop_duplicates(inplace=True)  # Remove duplicates in df2

# Ensure consistency in data types
df2["Revenue"] = pd.to_numeric(df2["Revenue"], errors="coerce")  # Convert 'Revenue' to numeric
df2["Cost"] = df2["Cost"].replace("[\$,]", "", regex=True).astype(float)  # Convert 'Cost' to numeric

# Convert 'Invoice Date' to proper datetime format
df2["Invoice Date"] = pd.to_datetime(df2["Invoice Date"], origin='1899-12-30', unit='D')

# Verify changes
df1.info(), df2.info(), df2.duplicated().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Primary Key                            500 non-null    object        
 1   Order No                               500 non-null    object        
 2   Segment Number                         500 non-null    int64         
 3   Order Date                             500 non-null    datetime64[ns]
 4   Manufacturer                           500 non-null    object        
 5   Model                                  500 non-null    object        
 6   Product Category                       500 non-null    object        
 7   Model Year                             500 non-null    int64         
 8   Serial Number                          500 non-null    object        
 9   Meter 1 Reading                        500 non-null    float64   

<ipython-input-13-180ba2f8b225>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1["Cause"].fillna("Unknown", inplace=True)  # Replace missing 'Cause' with 'Unknown'
<ipython-input-13-180ba2f8b225>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].

(None, None, 0)

##Data Integration
- Merge the two datasets on the identified primary key to create a comprehensive view of the datasets.
- Choose the appropriate type of join (inner, left, etc.), and justify your choice. Discuss the implications of using other join types in this context.


##Data Cleaning Summary

1. Handled Missing Values ->
Filled missing values in key columns like "Cause", "Correction", and "Coverage" with appropriate default values ("Unknown", "Not Provided", etc.).Set numerical columns ("Actual Hours", "Cost") to 0 where needed.

2. Removed Duplicates ->
Ensured "Primary Key" was unique by dropping duplicate rows.

3. Formatted Data Types ->
Converted "Cost" and "Segment Total $" to numeric (removed $ signs).
Converted "Invoice Date" to datetime format.Ensured "Primary Key" and "Order No" were stored as strings for consistent merging.

4. Merged Datasets ->
Used a LEFT JOIN on "Primary Key" to combine "Work Order Data" and "Repair Data", keeping all work orders even if repair data was missing.

##Why LEFT JOIN?
- Keeps all Work Order Data (df1) - Every work order remains, even if no matching repair record exists.
- Adds Repair Data (df2) when available - If a repair exists for the work order, it gets added.
- Prevents Data Loss - If we used an INNER JOIN, we'd lose work orders that don't have a repair record.

Implications of Other Join Types:

1. Inner Join: This join will only keep records that exist in both datasets. If a work order does not have a corresponding repair record, it will be excluded. This could lead to loss of important work order data.

2. Right Join: This join will keep all records from the Repair Data, even if there is no matching work order. This could introduce incomplete or orphaned repair records that may not be useful without the corresponding work order.

3. Full Outer Join: This join will keep all records from both datasets, even if they don't have a match. While this ensures that no data is lost, it could lead to unnecessary null values in many fields where there is no match between work orders and repairs, making the dataset harder to analyze

In [ ]:
pip install deep-translator

## Translated the correction column

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator

# # Load the Work Order Data
# file_path = "Data for Task 2..xlsx"  # Update with the correct path if needed
# df1 = pd.read_excel(file_path, sheet_name="Work Order Data")

# Function to detect and translate non-English text
def translate_text(text):
    try:
        if pd.notna(text):  # Check if text is not NaN
            return GoogleTranslator(source='auto', target='en').translate(text)
        return text
    except Exception:
        return text  # Return original text if translation fails

# Apply translation to the "Correction" column
df1["Correction"] = df1["Correction"].apply(translate_text)

# Save the translated dataset
output_file = "/content/drive/My Drive/Translated_Work_Order_Data.xlsx"
df1.to_excel(output_file, index=False)
print("Translation completed. File saved as Translated_Work_Order_Data.xlsx")

# df1.to_excel("Translated_Work_Order_Data.xlsx", index=False)
# print("Translation completed. File saved as Translated_Work_Order_Data.xlsx")



Translation completed. File saved as Translated_Work_Order_Data.xlsx


In [ ]:
import os
print("File Exists:", os.path.exists(file_path))
import pandas as pd

file_path = "/content/drive/My Drive/Translated_Work_Order_Data.xlsx"
df = pd.read_excel(file_path)

print(df.head())

File Exists: False
   Primary Key   Order No  Segment Number Order Date Manufacturer Model  \
0  SO0005588-1  SO0005588               1 2022-04-30       PASEIH  6780   
1  SO0005907-1  SO0005907               1 2022-04-30       PASEIH  6780   
2  SO0006100-1  SO0006100               1 2022-04-30       PASEIH  6780   
3  SO0006642-1  SO0006642               1 2022-04-30       PASEIH  6780   
4  SO0018457-1  SO0018457               1 2022-04-30       PASEIH  6780   

  Product Category  Model Year Serial Number  Meter 1 Reading  \
0             APPL           0     YFT042399        2531.0999   
1             APPL           0     YFT042399        2531.0999   
2             APPL           0     YFT042399        2531.0999   
3             APPL           0     YFT042399        2531.0999   
4             APPL           0     YFT042399        2531.0999   

                                           Complaint    Cause  \
0  No cab heat, temp gauge dont get to operating ...  Unknown   
1  No cab

## Merged the two datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
# Merge the datasets using a LEFT JOIN on "Primary Key"
df_merged = df1.merge(df2, on="Primary Key", how="left")

# Save the merged dataset
output_file = "/content/drive/My Drive/Merged_Complete_Dataset.csv"
df_merged.to_csv(output_file, index=False)
print(f"File saved to: {output_file}")


Mounted at /content/drive
File saved to: /content/drive/My Drive/Merged_Complete_Dataset.csv


## Filled the Cause Column

In [ ]:
df["Cause"] = df.apply(
    lambda row: f"{row['Complaint']} | {row['Failure Condition - Failure Component']} | {row['Fix Condition - Fix Component']}"
    if row["Cause"] == "Unknown" else row["Cause"], axis=1
)

# Save the updated file
df.to_excel("Cleaned_Merged_Data_Summarized_Task2.xlsx", index=False)

- Translated the Correction Column
- Merged the dataset
- Filled the Cause Column
- Total

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator

# Load the dataset
file_path = "Cleaned_Merged_Data_Summarized_Task2.xlsx"
df = pd.read_excel(file_path)

# Function to translate text to English
def translate_text(text):
    try:
        if pd.notna(text):  # Check if text is not NaN
            return GoogleTranslator(source='auto', target='en').translate(text)
        return text
    except Exception:
        return text  # Return original text if translation fails

# Apply translation to the "Correction" column
df["Correction"] = df["Correction"].apply(translate_text)

# Save the translated dataset as CSV
df.to_csv("Translated_Cleaned_Merged_Data.csv", index=False, encoding='utf-8')

print("Translation completed. File saved as Translated_Cleaned_Merged_Data.csv")


Translation completed. File saved as Translated_Cleaned_Merged_Data.csv
